In [ ]:
import importlib
import sys

from tqdm import tqdm
import llava.model.builder  # Import the module if not already imported
importlib.reload(llava.model.builder)
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
import torch
import os
import pandas as pd
import re
import argparse
from llava.constants import (
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
    IMAGE_PLACEHOLDER,
)
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import (
    process_images,
    tokenizer_image_token,
    get_model_name_from_path,
)
import argparse

from PIL import Image

import requests
from PIL import Image
from io import BytesIO
import re
from llava.eval.run_llava import image_parser, load_image, load_images



In [ ]:
# Function to evaluate the model for a given image and query

def evaluate_image(query, image_path):
    disable_torch_init()

    qs = query
    image_token_se = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN
    if IMAGE_PLACEHOLDER in qs:
        if model.config.mm_use_im_start_end:
            qs = re.sub(IMAGE_PLACEHOLDER, image_token_se, qs)
        else:
            qs = re.sub(IMAGE_PLACEHOLDER, DEFAULT_IMAGE_TOKEN, qs)
    else:
        if model.config.mm_use_im_start_end:
            qs = image_token_se + "\n" + qs
        else:
            qs = DEFAULT_IMAGE_TOKEN + "\n" + qs

    if "llama-2" in model_name.lower():
        conv_mode = "llava_llama_2"
    elif "mistral" in model_name.lower():
        print("using mistral conv mode...")
        conv_mode = "mistral_instruct"
    elif "v1.6-34b" in model_name.lower():
        conv_mode = "chatml_direct"
    elif "v1" in model_name.lower():
        conv_mode = "llava_v1"
    elif "mpt" in model_name.lower():
        conv_mode = "mpt"
    else:
        conv_mode = "llava_v0"

    if args.conv_mode is not None and conv_mode != args.conv_mode:
        print(
            "[WARNING] the auto inferred conversation mode is {}, while `--conv-mode` is {}, using {}".format(
                conv_mode, args.conv_mode, args.conv_mode
            )
        )
    else:
        args.conv_mode = conv_mode

    conv = conv_templates[args.conv_mode].copy()
    conv.append_message(conv.roles[0], qs)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()

    args.image_file = image_path
    image_files = image_parser(args)
    images = load_images(image_files)
    image_sizes = [x.size for x in images]
    images_tensor = process_images(
        images,
        image_processor,
        model.config
    ).to(model.device, dtype=torch.float16)

    input_ids = (
        tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
        .unsqueeze(0)
        .cuda()
    )

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=images_tensor,
            image_sizes=image_sizes,
            do_sample=True if args.temperature > 0 else False,
            temperature=args.temperature,
            top_p=args.top_p,
            num_beams=args.num_beams,
            max_new_tokens=args.max_new_tokens,
            use_cache=True,
        )

    outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()

    return outputs

def extract_dollar_amount(text):
   
   if not text:
       return None
   # Updated regex to optionally include cents after the decimal point
   matches = re.findall(r"\$(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)", text)
   if matches:
       # Remove commas and convert to float
       return float(matches[0].replace(',', ''))
   return None


### Drawer Zero-shot (whole)

In [ ]:

import sys
# Drawer class with wholeimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', 'liuhaotian/llava-v1.6-mistral-7b',
    # "--model-base", None,
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction
def extract_info(prediction):
    match = re.search(r'\"([^\"]+)[^\w\"]', prediction)
    return match.group(1) if match else None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/drawer.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):

    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test/{row['image_name']}"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Please give me the drawer on this check. Please use this answer template:The drawer on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'full_check_original_prediction'] = full_check_prediction
        df.loc[index, 'full_check_drawer_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

### Drawer Zero-shot (subimage)

In [ ]:

import sys
# Drawer class with subimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', 'liuhaotian/llava-v1.6-mistral-7b',
    # "--model-base", None,
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction and remove the trailing period if it exists
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    if match:
        extracted_info = match.group(1)
        if extracted_info.endswith('.'):
            extracted_info = extracted_info[:-1]  # Remove the trailing period
        return extracted_info
    return None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/drawer.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):
    # print(row)
    file_name = row['image_name'].split(".")[0]
    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/image_caption/test/{file_name}/drawer/subimage.png"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Please give me the drawer on this check. Please use this answer template:The drawer on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'subimage_original_prediction'] = full_check_prediction
        
        df.loc[index, 'subimage_drawer_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

### Drawer LoRA (whole)

In [ ]:

import sys
# Drawer class with wholeimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', '/home/larry5/project/LLaVA-1.6-ft/scripts_peft/mistral/lora/llava-lora-mistral-r128a256/wholeimage/drawer/llava-lora-mistral-r128a256-10BS-model',
    "--model-base", "liuhaotian/llava-v1.6-mistral-7b",
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction
def extract_info(prediction):
    match = re.search(r'\"([^\"]+)[^\w\"]', prediction)
    return match.group(1) if match else None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/drawer.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):

    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test/{row['image_name']}"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        # full_check_prediction = evaluate_image("Please give me the drawer on this check. Please use this answer template:The drawer on this check is \"xxx.\"", full_check_path)
        full_check_prediction = evaluate_image("Please give me the drawer on this check.", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'lora_full_check_original_prediction'] = full_check_prediction
        df.loc[index, 'lora_full_check_drawer_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

### Drawer LoRA (subimage)

In [ ]:

import sys
# Drawer class with subimage lora 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', '/home/larry5/project/LLaVA-1.6-ft/scripts_peft/mistral/lora/llava-lora-mistral-r128a256/subimage/drawer/llava-lora-mistral-r128a256-10BS-model',
    "--model-base", 'liuhaotian/llava-v1.6-mistral-7b',
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction and remove the trailing period if it exists
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    if match:
        extracted_info = match.group(1)
        if extracted_info.endswith('.'):
            extracted_info = extracted_info[:-1]  # Remove the trailing period
        return extracted_info
    return None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/drawer.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):
    # print(row)
    file_name = row['image_name'].split(".")[0]
    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/image_caption/test/{file_name}/drawer/subimage.png"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Please give me the drawer on this check. Please use this answer template:The drawer on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'lora_subimage_original_prediction'] = full_check_prediction
        
        df.loc[index, 'lora_subimage_drawer_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

### Bank_no Zero-shot (whole)

In [ ]:

import sys
# bank_no class with wholeimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', 'liuhaotian/llava-v1.6-mistral-7b',
    # "--model-base", None,
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction
def extract_info(prediction):
    match = re.search(r'\"([^\"]+)[^\w\"]', prediction)
    return match.group(1) if match else None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/bank_no.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):

    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test/{row['image_name']}"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Please give me the bank number on this check. Please use this answer template:The bank number on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'full_check_original_prediction'] = full_check_prediction
        df.loc[index, 'full_check_bank_no_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

### Bank_no Zero-shot (subimage)

In [9]:

import sys
# bank_no class with subimage zero-shot 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', 'liuhaotian/llava-v1.6-mistral-7b',
    # "--model-base", None,
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction
def extract_info(prediction):
    match = re.search(r'\"([^\"]+)[^\w\"]', prediction)
    return match.group(1) if match else None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/bank_no.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):
    # print(row)
    file_name = row['image_name'].split(".")[0]
    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/image_caption/test/{file_name}/back_no/subimage.png"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Please give me the bank number on this check. Please use this answer template:The bank number on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'subimage_original_prediction'] = full_check_prediction
        df.loc[index, 'subimage_bank_no_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

Image file: /home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png
Updated image file: Test.png


/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.class_embedding: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.patch_embedding.weight: copying from a non

using mistral conv mode...


2it [00:00,  2.73it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The bank number on this check is "5-70017/60110."
using mistral conv mode...


...

The bank number on this check is "565-2/212 88859 7374701118."
using mistral conv mode...


503it [12:15,  1.33it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The bank number on this check is "644-00228860-6010."
using mistral conv mode...


504it [12:16,  1.46s/it]

The bank number on this check is "332-661/11111110."


### Bank_no LoRA (whole)

In [15]:

import sys
# bank no class with wholeimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', '/home/larry5/project/LLaVA-1.6-ft/scripts_peft/mistral/lora/llava-lora-mistral-r128a256/wholeimage/bank_no/llava-lora-mistral-r128a256-10BS-model',
    "--model-base", "liuhaotian/llava-v1.6-mistral-7b",
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction
def extract_info(prediction):
    match = re.search(r'\"([^\"]+)[^\w\"]', prediction)
    return match.group(1) if match else None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/bank_no.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):

    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test/{row['image_name']}"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        # full_check_prediction = evaluate_image("Please give me the drawer on this check. Please use this answer template:The drawer on this check is \"xxx.\"", full_check_path)
        full_check_prediction = evaluate_image("Please give me the bank number on this check. Please use this answer template:The bank number on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'lora_full_check_original_prediction'] = full_check_prediction
        df.loc[index, 'lora_full_check_bank_no_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

Image file: /home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png
Updated image file: Test.png


/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.class_embedding: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.patch_embedding.weight: copying from a non

Loading LLaVA from base model...
Loading additional LLaVA weights...
Loading LoRA weights...
Merging LoRA weights...
Model is loaded...


0it [00:00, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


using mistral conv mode...


...

The bank number on this check is "132/23 13200".
using mistral conv mode...


6it [00:05,  1.11it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The bank number on this check is "466 Onderdonk Ave 1-R 1385".
using mistral conv mode...


6it [00:05,  1.00it/s]


KeyboardInterrupt: 

### Bank_no LoRA (subimage)

In [16]:

import sys
# bank no class with wholeimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', '/home/larry5/project/LLaVA-1.6-ft/scripts_peft/mistral/lora/llava-lora-mistral-r128a256/subimage/bank_no/llava-lora-mistral-r128a256-10BS-model',
    "--model-base", "liuhaotian/llava-v1.6-mistral-7b",
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    return match.group(1) if match else None
    

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/bank_no.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):
    file_name = row['image_name'].split(".")[0]
    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/image_caption/test/{file_name}/back_no/subimage.png"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        # full_check_prediction = evaluate_image("Please give me the drawer on this check. Please use this answer template:The drawer on this check is \"xxx.\"", full_check_path)
        full_check_prediction = evaluate_image("Please give me the bank number on this check. Please use this answer template:The bank number on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'lora_subimage_original_prediction'] = full_check_prediction
        df.loc[index, 'lora_subimage_bank_no_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

Image file: /home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png
Updated image file: Test.png


/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.class_embedding: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.patch_embedding.weight: copying from a non

Loading LLaVA from base model...
Loading additional LLaVA weights...
Loading LoRA weights...
Merging LoRA weights...
Model is loaded...


0it [00:00, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


using mistral conv mode...


...

The bank number on this check is "55-2/212 8859 7397470118".
using mistral conv mode...


503it [05:15,  1.45it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The bank number on this check is "64-9238/610".
using mistral conv mode...


504it [05:16,  1.59it/s]

The bank number on this check is "32-61/1110 013".


In [12]:
import pandas as pd
import re

# Function to extract information from prediction
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    return match.group(1) if match else None
    
# Define the CSV path and the column names
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/bank_no.csv"
original_prediction_col = "lora_full_check_original_prediction"
extracted_info_col = "lora_full_check_bank_no_prediction"

# Load the CSV file
df = pd.read_csv(csv_path)

# Apply the extraction function to the prediction column
df[extracted_info_col] = df[original_prediction_col].apply(lambda x: extract_info(x))

# Save the updated DataFrame back to the CSV file
df.to_csv(csv_path, index=False)

print(f"Extracted information saved to {csv_path}")


Extracted information saved to /home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/bank_no.csv


In [ ]:
import pandas as pd
import re



# Function to extract information from prediction and remove the trailing period if it exists
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    if match:
        extracted_info = match.group(1)
        if extracted_info.endswith('.'):
            extracted_info = extracted_info[:-1]  # Remove the trailing period
        return extracted_info
    return None
# Define the CSV path and the column names
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/bank_no.csv"
original_prediction_col = "full_check_original_prediction"
extracted_info_col = "full_check_bank_no_prediction"

# Load the CSV file
df = pd.read_csv(csv_path)

# Apply the extraction function to the prediction column
df[extracted_info_col] = df[original_prediction_col].apply(lambda x: extract_info(x))

# Save the updated DataFrame back to the CSV file
df.to_csv(csv_path, index=False)

print(f"Extracted information saved to {csv_path}")


### check_no Zero-Shot (whole)

In [ ]:

import sys
# check number class with wholeimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', 'liuhaotian/llava-v1.6-mistral-7b',
    # "--model-base", None,
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction and remove the trailing period if it exists
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    if match:
        extracted_info = match.group(1)
        if extracted_info.endswith('.'):
            extracted_info = extracted_info[:-1]  # Remove the trailing period
        return extracted_info
    return None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/check_no.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):

    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test/{row['image_name']}"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Please give me the check number on this check. Please use this answer template:The check number on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'full_check_original_prediction'] = full_check_prediction
        df.loc[index, 'full_check_check_no_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

### check_no Zero-Shot (subimage)

In [17]:

import sys
# check number class with subimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', 'liuhaotian/llava-v1.6-mistral-7b',
    # "--model-base", None,
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction and remove the trailing period if it exists
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    if match:
        extracted_info = match.group(1)
        if extracted_info.endswith('.'):
            extracted_info = extracted_info[:-1]  # Remove the trailing period
        return extracted_info
    return None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/check_no.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):
    file_name = row['image_name'].split(".")[0]
    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/image_caption/test/{file_name}/check_no/subimage.png"
 
    # full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test/{row['image_name']}"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Please give me the check number on this check. Please use this answer template:The check number on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'subimage_original_prediction'] = full_check_prediction
        df.loc[index, 'subimage_check_no_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

Image file: /home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png
Updated image file: Test.png


/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.class_embedding: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.patch_embedding.weight: copying from a non

using mistral conv mode...


1it [00:00,  2.11it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The check number on this check is "220."
using mistral conv mode...


...

The check number on this check is "214."
using mistral conv mode...


503it [05:05,  1.94it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The check number on this check is "3082260".
using mistral conv mode...


504it [05:06,  1.65it/s]

The check number on this check is "65538."


### check_no LoRA (whole)

In [ ]:

import sys
# check no class with wholeimage lora

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', "/home/larry5/project/LLaVA-1.6-ft/scripts_peft/mistral/lora/llava-lora-mistral-r128a256/wholeimage/check_no/llava-lora-mistral-r128a256-10BS-model",
    "--model-base", 'liuhaotian/llava-v1.6-mistral-7b',
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction and remove the trailing period if it exists
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    if match:
        extracted_info = match.group(1)
        if extracted_info.endswith('.'):
            extracted_info = extracted_info[:-1]  # Remove the trailing period
        return extracted_info
    return None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/check_no.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):

    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test/{row['image_name']}"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Please give me the check number on this check. Please use this answer template:The check number on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'lora_full_check_original_prediction'] = full_check_prediction
        df.loc[index, 'lora_full_check_check_no_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

### check_no LoRA (subimage)

In [18]:

import sys
# check no class with subimage lora

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', "/home/larry5/project/LLaVA-1.6-ft/scripts_peft/mistral/lora/llava-lora-mistral-r128a256/subimage/check_no/llava-lora-mistral-r128a256-10BS-model",
    "--model-base", 'liuhaotian/llava-v1.6-mistral-7b',
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction and remove the trailing period if it exists
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    if match:
        extracted_info = match.group(1)
        if extracted_info.endswith('.'):
            extracted_info = extracted_info[:-1]  # Remove the trailing period
        return extracted_info
    return None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/check_no.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):
    file_name = row['image_name'].split(".")[0]
    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/image_caption/test/{file_name}/check_no/subimage.png"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Please give me the check number on this check. Please use this answer template:The check number on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'lora_subimage_original_prediction'] = full_check_prediction
        df.loc[index, 'lora_subimage_check_no_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

Image file: /home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png
Updated image file: Test.png


/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.class_embedding: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.patch_embedding.weight: copying from a non

Loading LLaVA from base model...
Loading additional LLaVA weights...
Loading LoRA weights...
Merging LoRA weights...
Model is loaded...


0it [00:00, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


using mistral conv mode...


...

The check number on this check is "214".
using mistral conv mode...


503it [04:16,  1.96it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The check number on this check is "308260".
using mistral conv mode...


504it [04:16,  1.96it/s]

The check number on this check is "6558".


### Bank Name Zero-shot (whole)

In [ ]:

import sys
# bank name class with wholeimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', 'liuhaotian/llava-v1.6-mistral-7b',
    # "--model-base", None,
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction and remove the trailing period if it exists
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    if match:
        extracted_info = match.group(1)
        if extracted_info.endswith('.'):
            extracted_info = extracted_info[:-1]  # Remove the trailing period
        return extracted_info
    return None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/bank.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):

    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test/{row['image_name']}"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Please give me the bank name on this check. Please use this answer template:The bank name on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'full_check_original_prediction'] = full_check_prediction
        df.loc[index, 'full_check_bank_name_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

### Bank Name Zero-shot (subimage)

In [19]:

import sys
# bank name class with subimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', 'liuhaotian/llava-v1.6-mistral-7b',
    # "--model-base", None,
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction and remove the trailing period if it exists
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    if match:
        extracted_info = match.group(1)
        if extracted_info.endswith('.'):
            extracted_info = extracted_info[:-1]  # Remove the trailing period
        return extracted_info
    return None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/bank.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):
    file_name = row['image_name'].split(".")[0]
    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/image_caption/test/{file_name}/bank/subimage.png"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Please give me the bank name on this check. Please use this answer template:The bank name on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'subimage_original_prediction'] = full_check_prediction
        df.loc[index, 'subimage_bank_name_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

Image file: /home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png
Updated image file: Test.png


/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.class_embedding: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.patch_embedding.weight: copying from a non

using mistral conv mode...


1it [00:00,  2.20it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The bank name on this check is "Chase."
using mistral conv mode...


...

The bank name on this check is "Wells Fargo Bank, N.A."
using mistral conv mode...


503it [04:09,  1.76it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The bank name on this check is "JPMORGAN CHASE BANK, N.A."
using mistral conv mode...


504it [04:09,  2.02it/s]

The bank name on this check is "CHASE."


### Bank Name LoRA (whole)

In [ ]:

import sys
# bank name class with wholeimage lora

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', '/home/larry5/project/LLaVA-1.6-ft/scripts_peft/mistral/lora/llava-lora-mistral-r128a256/wholeimage/bank/llava-lora-mistral-r128a256-10BS-model',
    "--model-base", 'liuhaotian/llava-v1.6-mistral-7b',
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction and remove the trailing period if it exists
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    if match:
        extracted_info = match.group(1)
        if extracted_info.endswith('.'):
            extracted_info = extracted_info[:-1]  # Remove the trailing period
        return extracted_info
    return None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/bank.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):

    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test/{row['image_name']}"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Please give me the bank name on this check. Please use this answer template:The bank name on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'lora_full_check_original_prediction'] = full_check_prediction
        df.loc[index, 'lora_full_check_bank_name_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

### Bank Name LoRA (subimage)

In [20]:

import sys
# bank name class with subimage lora

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', '/home/larry5/project/LLaVA-1.6-ft/scripts_peft/mistral/lora/llava-lora-mistral-r128a256/subimage/bank/llava-lora-mistral-r128a256-10BS-model',
    "--model-base", 'liuhaotian/llava-v1.6-mistral-7b',
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Function to extract information from prediction and remove the trailing period if it exists
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    if match:
        extracted_info = match.group(1)
        if extracted_info.endswith('.'):
            extracted_info = extracted_info[:-1]  # Remove the trailing period
        return extracted_info
    return None

# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/bank.csv"
df = pd.read_csv(csv_path)

# Processing each entry in the CSV
for index, row in tqdm(df.iterrows()):
    file_name = row['image_name'].split(".")[0]
    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/image_caption/test/{file_name}/bank/subimage.png"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Please give me the bank name on this check. Please use this answer template:The bank name on this check is \"xxx.\"", full_check_path)
        print(full_check_prediction)
        # Extract and store dollar amounts in new columns
        df.loc[index, 'lora_subimage_original_prediction'] = full_check_prediction
        df.loc[index, 'lora_subimage_bank_name_prediction'] = extract_info(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

Image file: /home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png
Updated image file: Test.png


/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.class_embedding: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/home/larry5/miniconda3/envs/llava/lib/python3.10/site-packages/torch/nn/modules/module.py:2025: UserWarning: for vision_model.embeddings.patch_embedding.weight: copying from a non

Loading LLaVA from base model...
Loading additional LLaVA weights...
Loading LoRA weights...
Merging LoRA weights...
Model is loaded...


0it [00:00, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


using mistral conv mode...


...

The bank name on this check is "Wells Fargo".
using mistral conv mode...


503it [04:26,  1.79it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The bank name on this check is "JPMorgan Chase Bank,N.A".
using mistral conv mode...


504it [04:26,  1.89it/s]

The bank name on this check is "CHASE".


In [ ]:
# Update Drawer cause we missed some values while converting the format

In [ ]:
import pandas as pd
import re

# Function to extract information from prediction
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    return match.group(1) if match else None
    
# Define the CSV path and the column names
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/drawer.csv"
original_prediction_col = "lora_full_check_original_prediction"
extracted_info_col = "lora_full_check_drawer_prediction"

# Load the CSV file
df = pd.read_csv(csv_path)

# Apply the extraction function to the prediction column
df[extracted_info_col] = df[original_prediction_col].apply(lambda x: extract_info(x))

# Save the updated DataFrame back to the CSV file
df.to_csv(csv_path, index=False)

print(f"Extracted information saved to {csv_path}")


In [ ]:
import pandas as pd
import re



# Function to extract information from prediction and remove the trailing period if it exists
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    if match:
        extracted_info = match.group(1)
        if extracted_info.endswith('.'):
            extracted_info = extracted_info[:-1]  # Remove the trailing period
        return extracted_info
    return None
# Define the CSV path and the column names
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm_other_class/drawer.csv"
original_prediction_col = "full_check_original_prediction"
extracted_info_col = "full_check_drawer_prediction"

# Load the CSV file
df = pd.read_csv(csv_path)

# Apply the extraction function to the prediction column
df[extracted_info_col] = df[original_prediction_col].apply(lambda x: extract_info(x))

# Save the updated DataFrame back to the CSV file
df.to_csv(csv_path, index=False)

print(f"Extracted information saved to {csv_path}")


### Amount class: subimage and wholeimage

In [ ]:
import sys
# Amount class with subimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', 'liuhaotian/llava-v1.6-mistral-7b',
    # "--model-base", None,
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the dollar amount on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)



# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm/updated_final_predictions_amount.csv"
df = pd.read_csv(csv_path)

# Processing only the rows where 'lora_full_check_amount_prediction' is NaN
na_count_before = df['subimage_original_prediction'].isna().sum()
print(f"Number of NaN entries before processing: {na_count_before}")

for index, row in tqdm(df[df['subimage_original_prediction'].isna()].iterrows(), total=na_count_before):
    naive_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/image_caption_test_baidu/test/{row['image_name'].split('.')[0]}/amount/subimage.png"
    # full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test{row['image_name']}"
    # print(naive_path)
    if os.path.isfile(naive_path):
    # Evaluate the model for naive and full check images
        naive_prediction = evaluate_image("Can you tell me the dollar amount on this check.", naive_path)
        # full_check_prediction = evaluate_image("Can you tell me the dollar amount on this check.", full_check_path)

        # Extract and store dollar amounts in new columns
        df.loc[index, 'subimage_original_prediction'] = naive_prediction
        # df.loc[index, 'lora_full_check_original_prediction'] = full_check_prediction
        df.loc[index, 'subimage_amount_prediction'] = extract_dollar_amount(naive_prediction)
        # df.loc[index, 'lora_full_check_amount_prediction'] = extract_dollar_amount(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv('/home/larry5/project/LLaVA-1.6-ft/data/cikm/updated_final_predictions_amount.csv', index=False)

In [ ]:
import sys
# Amount class with wholeimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', 'liuhaotian/llava-v1.6-mistral-7b',
    # "--model-base", None,
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the dollar amount on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)



# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm/updated_final_predictions_amount.csv"
df = pd.read_csv(csv_path)

# Processing only the rows where 'lora_full_check_amount_prediction' is NaN
na_count_before = df['full_check_original_prediction'].isna().sum()
print(f"Number of NaN entries before processing: {na_count_before}")

for index, row in tqdm(df[df['full_check_original_prediction'].isna()].iterrows(), total=na_count_before):

    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test/{row['image_name']}"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Can you tell me the dollar amount on this check.", full_check_path)

        # Extract and store dollar amounts in new columns
        df.loc[index, 'full_check_original_prediction'] = full_check_prediction
        df.loc[index, 'full_check_amount_prediction'] = extract_dollar_amount(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

### Payee class with subimage and wholeimage

In [ ]:
import sys
# Payee class with subimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', 'liuhaotian/llava-v1.6-mistral-7b',
    # "--model-base", None,
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)

# Function to extract information from prediction
def extract_info(prediction):
    match = re.search(r'\"([^\"]+)[^\w\"]', prediction)
    return match.group(1) if match else None
# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm/updated_final_predictions_payee.csv"
df = pd.read_csv(csv_path)

# Processing only the rows where 'lora_full_check_amount_prediction' is NaN
na_count_before = df['subimage_original_prediction'].isna().sum()
print(f"Number of NaN entries before processing: {na_count_before}")

for index, row in tqdm(df[df['subimage_original_prediction'].isna()].iterrows(), total=na_count_before):

    naive_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/image_caption_test_baidu/test/{row['image_name'].split('.')[0]}/payee/subimage.png"
    # full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test{row['image_name']}"
    # print(naive_path)
    if os.path.isfile(naive_path):
    # Evaluate the model for naive and full check images
        naive_prediction = evaluate_image("Can you tell me the payee on this check.", naive_path)
        # full_check_prediction = evaluate_image("Can you tell me the dollar amount on this check.", full_check_path)

        # Extract and store dollar amounts in new columns
        df.loc[index, 'subimage_original_prediction'] = naive_prediction
        # df.loc[index, 'lora_full_check_original_prediction'] = full_check_prediction
        df.loc[index, 'subimage_payee_prediction'] = extract_info(naive_prediction)
        # df.loc[index, 'lora_full_check_amount_prediction'] = extract_dollar_amount(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

In [ ]:
# reslove the extract info function
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    return match.group(1) if match else None

# Load the CSV file
df = pd.read_csv(csv_path)

# Update 'lora_subimage_payee_prediction' based on 'lora_subimage_original_prediction'
for index, row in df.iterrows():
    if pd.isna(row['subimage_original_prediction']):
        continue
    original_prediction = row['subimage_original_prediction']
    payee_prediction = extract_info(original_prediction)
    df.loc[index, 'subimage_payee_prediction'] = payee_prediction

# Save the updated DataFrame to the same or a new CSV file
df.to_csv(csv_path, index=False)

In [ ]:

import sys
# Payee class with wholeimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', 'liuhaotian/llava-v1.6-mistral-7b',
    # "--model-base", None,
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the payee on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)


# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm/updated_final_predictions_payee.csv"
df = pd.read_csv(csv_path)

# Processing only the rows where 'lora_full_check_amount_prediction' is NaN
na_count_before = df['full_check_original_prediction'].isna().sum()
print(f"Number of NaN entries before processing: {na_count_before}")

for index, row in tqdm(df[df['full_check_original_prediction'].isna()].iterrows(), total=na_count_before):

    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test/{row['image_name']}"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        full_check_prediction = evaluate_image("Can you tell me the payee on this check.", full_check_path)

        # Extract and store dollar amounts in new columns
        df.loc[index, 'full_check_original_prediction'] = full_check_prediction
        df.loc[index, 'full_check_payee_prediction'] = extract_dollar_amount(full_check_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

In [ ]:
# reslove the extract info function 
# otherwise the extract_info won't work properly 
def extract_info(prediction):
    match = re.search(r'"([^"]+)"', prediction)
    return match.group(1) if match else None

# Load the CSV file
df = pd.read_csv(csv_path)

# Update 'lora_subimage_payee_prediction' based on 'lora_subimage_original_prediction'
for index, row in df.iterrows():
    if pd.isna(row['full_check_original_prediction']):
        continue
    original_prediction = row['full_check_original_prediction']
    payee_prediction = extract_info(original_prediction)
    df.loc[index, 'full_check_payee_prediction'] = payee_prediction

# Save the updated DataFrame to the same or a new CSV file
df.to_csv(csv_path, index=False)

### Date class for subimage and wholeimage

In [ ]:
#update the pattern matching to ungroup the month day year
def extract_date_from_response(text):
    import re
    description = []  # List to store descriptions of what padding was applied

    # Regex for month, day, and year, independently
    month_match = re.search(r"month: (\d{1,2})", text)
    day_match = re.search(r"day: (\d{1,2})", text)
    year_match = re.search(r"year: (\d{1,4})", text)

    # Process month
    if month_match:
        month = month_match.group(1)
        if len(month) < 2:
            month = month.zfill(2)
            description.append(f"Month padded to two digits: {month}")
        month = int(month)
    else:
        month = None
        description.append("Month not detected.")

    # Process day
    if day_match:
        day = day_match.group(1)
        if len(day) < 2:
            day = day.zfill(2)
            description.append(f"Day padded to two digits: {day}")
        day = int(day)
    else:
        day = None
        description.append("Day not detected.")

    # Process year
    if year_match:
        year = year_match.group(1)
        if len(year) == 1:
            year = "202" + year
            description.append(f"Year padded from one digit to four digits: {year}")
        elif len(year) == 2:
            year = "20" + year
            description.append(f"Year padded from two digits to four digits: {year}")
        elif len(year) == 3:
            year = "2" + year
            description.append(f"Year padded from three digits to four digits: {year}")
        year = int(year)
    else:
        year = None
        description.append("Year not detected.")

    return month, day, year
    # return month, day, year, " | ".join(description)

# Example usage
response_text = "The check was issued on month: 9, year: 21."
month, day, year = extract_date_from_response(response_text)
print(f"Month: {month}, Day: {day}, Year: {year}")
# print("Description:", padding_description)


In [ ]:
import sys
# Date class with subimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', 'liuhaotian/llava-v1.6-mistral-7b',
    # "--model-base", None,
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the dollar amount on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)



In [ ]:


# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm/updated_final_predictions_date.csv"
df = pd.read_csv(csv_path)


# Processing only the rows where 'lora_full_check_amount_prediction' is NaN
na_count_before = df['original_prediction'].isna().sum()
print(f"Number of NaN entries before processing: {na_count_before}")

for index, row in tqdm(df[df['original_prediction'].isna()].iterrows(), total=na_count_before):

    naive_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/image_caption_test_baidu/test/{row['image_name'].split('.')[0]}/date/subimage.png"
    # full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test{row['image_name']}"
    # print(naive_path)
    print(naive_path, os.path.isfile(naive_path))
    if os.path.isfile(naive_path):
    # Evaluate the model for naive and full check images
        prompt = "Based on the format below, identify the date from the provided check image:\n    1. Month of issuance (MM)\n    2. Day of issuance (DD)\n    3. Year of issuance (YYYY)\n\n    Please provide your answer only using the given context and format it as follows:\n    'The check was issued on month: MM, day: DD, year: YYYY.'"        
        naive_prediction = evaluate_image(prompt, naive_path)
        # full_check_prediction = evaluate_image("Can you tell me the dollar amount on this check.", full_check_path)

        # Extract and store dollar amounts in new columns
        df.loc[index, 'original_prediction'] = naive_prediction
        df.loc[index, 'month_prediction'],df.loc[index, 'day_prediction'], df.loc[index, 'year_prediction']  = extract_date_from_response(naive_prediction)
    
# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)

In [ ]:
import sys
# Date class with wholeimage 

# variables need to be modified:
# 1. model-path
# 2. csv-path
# 3. image-path



# Simulating command line arguments (as if you were running this from a terminal)
sys.argv = [
    'script',  # Typically, sys.argv[0] is the script name, it can be anything.
    '--model-path', 'liuhaotian/llava-v1.6-mistral-7b',
    # "--model-base", None,
    '--image-file', '/home/larry5/project/LLaVA-1.6-ft/data/03/dataset/images/test/photo_7403@24-03-2022_04-19-03_1.png',
    '--query', 'Can you tell me the dollar amount on this check.',
    '--temperature', '0.5'
]

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, default=None)
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=None, required=False)
parser.add_argument("--query", type=str, required=True)
parser.add_argument("--conv-mode", type=str, default=None)
parser.add_argument("--sep", type=str, default=",")
parser.add_argument("--temperature", type=float, default=0.2)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=512)
args = parser.parse_args()
print("Image file:", args.image_file)
args.image_file = "Test.png"
print("Updated image file:", args.image_file)


# Load model and tokenizer
model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name)



# Load CSV file 
csv_path = "/home/larry5/project/LLaVA-1.6-ft/data/cikm/updated_final_predictions_date.csv"
df = pd.read_csv(csv_path)


# Processing only the rows where 'lora_full_check_amount_prediction' is NaN
na_count_before = df['full_check_original_prediction'].isna().sum()
print(f"Number of NaN entries before processing: {na_count_before}")

for index, row in tqdm(df[df['full_check_original_prediction'].isna()].iterrows(), total=na_count_before):
    full_check_path = f"/home/larry5/project/LLaVA-1.6-ft/data/data/03/dataset/images/test/{row['image_name']}"
    if os.path.isfile(full_check_path):
    # Evaluate the model for naive and full check images
        prompt = "Based on the format below, identify the date from the provided check image:\n    1. Month of issuance (MM)\n    2. Day of issuance (DD)\n    3. Year of issuance (YYYY)\n\n    Please provide your answer only using the given context and format it as follows:\n    'The check was issued on month: MM, day: DD, year: YYYY.'"        

        full_check_prediction = evaluate_image(prompt, full_check_path)

        # Extract and store dollar amounts in new columns
        df.loc[index, 'full_check_original_prediction'] = full_check_prediction
        df.loc[index, 'full_check_month_prediction'], df.loc[index, 'full_check_day_prediction'], df.loc[index, 'full_check_year_prediction']= extract_date_from_response(full_check_prediction)

# Save the updated DataFrame to a new CSV
df.to_csv(csv_path, index=False)